## Финальное соревнование

##### Автор: [Радослав Нейчев](https://www.linkedin.com/in/radoslav-neychev/), @neychev

В данном задании вас ждет неизвестная зависимость. Ваша основная задача: **построить две лучших модели**, минимизирующих среднеквадратичную ошибку (MSE):
1. На первую модель не налагается ограничений.
2. Вторая модель должна быть **линейной**, т.е. представлять собой линейную комбинацию признаков плюс свободный член: $\boldsymbol{w}^{\top}\boldsymbol{x} + b$. При этом __вы можете использовать базовые математические операции для преобразования признаков__: np.exp, np.log, np.pow (полный список доступен в [документации](https://numpy.org/doc/stable/reference/routines.math.html)), а также линейные операции над ними (сумма, умножение на число и пр.). Для преобразования признаков вам будет необходимо написать функцию `my_transformation`. __Кол-во параметров (весов) используемых второй моделью не должно превышать 15 (включая свободный член).__

Настоятельно рекомендуем написать код "с нуля", лишь поглядывая на готовые примеры, а не просто "скопировать-вставить". Это поможет вам в будущем писать код более уверенно

In [58]:
import os
import json

import numpy as np
from matplotlib import pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

Загрузка данных происходит ниже. Если она не срабатывает, самостоятельно скачайте файл `hw_final_open_data.npy` и положите его в ту же директорию, что и ноутбук.

In [39]:
!wget https://raw.githubusercontent.com/girafe-ai/ml-course/23f_yandex_ml_trainings/homeworks/assignment_final/hw_final_open_data.npy -O hw_final_open_data.npy
!wget https://raw.githubusercontent.com/girafe-ai/ml-course/23f_yandex_ml_trainings/homeworks/assignment_final/hw_final_open_target.npy -O hw_final_open_target.npy

--2023-11-30 07:53:45--  https://raw.githubusercontent.com/girafe-ai/ml-course/23f_yandex_ml_trainings/homeworks/assignment_final/hw_final_open_data.npy
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44928 (44K) [application/octet-stream]
Saving to: ‘hw_final_open_data.npy’

hw_final_open_data. 100%[===================>]  43.88K  --.-KB/s    in 0.009s  

2023-11-30 07:53:45 (4.91 MB/s) - ‘hw_final_open_data.npy’ saved [44928/44928]

--2023-11-30 07:53:45--  https://raw.githubusercontent.com/girafe-ai/ml-course/23f_yandex_ml_trainings/homeworks/assignment_final/hw_final_open_target.npy
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubus

In [40]:
assert os.path.exists('hw_final_open_data.npy'), 'Please, download `hw_final_open_data.npy` and place it in the working directory'
assert os.path.exists('hw_final_open_target.npy'), 'Please, download `hw_final_open_target.npy` and place it in the working directory'
data = np.load('hw_final_open_data.npy', allow_pickle=False)
target = np.load('hw_final_open_target.npy', allow_pickle=False)

In [44]:
data.dtype, target.dtype

(dtype('float64'), dtype('float64'))

In [45]:
data.shape, target.shape

((800, 7), (800,))

In [ ]:
!pip install catboost

In [114]:
from catboost import CatBoostRegressor, Pool

In [164]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

In [229]:
from scipy import stats

model = CatBoostRegressor(
    random_seed=42,
    thread_count=-1,
    eval_metric="MAPE",
    verbose=500,
)

param_distribution = {
    "one_hot_max_size": stats.bernoulli(p=0.2, loc=2),
    "learning_rate": [0.01, 0.02, 0.03, 0.04, 0.05],
    "l2_leaf_reg": [1, 2, 3, 4],
    "depth": stats.binom(n=10, p=0.2),
}
randomized_search_result = model.randomized_search(param_distribution, X_train, y_train)
model.best_score_

0:	learn: 0.9168623	test: 0.9162263	best: 0.9162263 (0)	total: 648us	remaining: 648ms
500:	learn: 0.0918676	test: 0.1000051	best: 0.1000051 (500)	total: 101ms	remaining: 100ms
999:	learn: 0.0910246	test: 0.0999736	best: 0.0999367 (776)	total: 188ms	remaining: 0us

bestTest = 0.09993665672
bestIteration = 776

0:	loss: 0.0999367	best: 0.0999367 (0)	total: 207ms	remaining: 1.86s
0:	learn: 0.9067114	test: 0.9058744	best: 0.9058744 (0)	total: 1.46ms	remaining: 1.46s
500:	learn: 0.0865155	test: 0.0978275	best: 0.0977329 (418)	total: 228ms	remaining: 227ms
999:	learn: 0.0815044	test: 0.0957670	best: 0.0956921 (987)	total: 453ms	remaining: 0us

bestTest = 0.09569209836
bestIteration = 987

1:	loss: 0.0956921	best: 0.0956921 (1)	total: 677ms	remaining: 2.71s
0:	learn: 0.9277909	test: 0.9273902	best: 0.9273902 (0)	total: 274us	remaining: 274ms
500:	learn: 0.0933098	test: 0.0992498	best: 0.0992475 (498)	total: 94.8ms	remaining: 94.4ms
999:	learn: 0.0916083	test: 0.0979548	best: 0.0979402 (987)	t

{'learn': {'MAPE': 0.08625366593225722, 'RMSE': 0.2730106597497571}}

In [169]:
randomized_search_result["params"]

{'l2_leaf_reg': 3,
 'depth': 4.0,
 'one_hot_max_size': 2.0,
 'learning_rate': 0.04}

In [285]:
params = randomized_search_result["params"]

model = CatBoostRegressor(eval_metric='RMSE', **params, random_state=174)
model.fit(X_train, y_train, verbose=500)

0:	learn: 2.1035182	total: 504us	remaining: 504ms
500:	learn: 0.2826089	total: 252ms	remaining: 251ms
999:	learn: 0.2718705	total: 497ms	remaining: 0us


In [263]:
def error(y_true, y_pred):
    print(mean_squared_error(y_true, y_pred))

In [286]:
error(y_test, model.predict(X_test))

0.10425905934748651


Разбивка на `train` и `val` опциональна и сделана для вашего удобства.

In [388]:
train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.2, random_state=551)

### Модель №1
Напоминаем, в первой части задания ваша основная задача – получить наилучший результат без ограничений на модель. Сдаваться будут только предсказания модели.

Пример с использованием Random Forest доступен ниже.

In [282]:
rf = RandomForestRegressor(random_state=290)
rf.fit(train_x, train_y)

print(
    f'train mse =\t {mean_squared_error(np.round(rf.predict(train_x), 2), np.round(train_y)):.5f}',
    f'validation mse = {mean_squared_error(np.round(rf.predict(valid_x)), np.round(valid_y)):.5f}',
    sep='\n'
)

train mse =	 0.10502
validation mse = 0.27500


In [325]:
lr = Ridge()
mse = 0.06869
for state in range(0, 1001):
    train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.2, random_state=state)
    lr.fit(train_x, train_y)
    new_mse = mean_squared_error(lr.predict(valid_x), valid_y)
    if new_mse < mse:
        print(state)
        mse = new_mse

43
144
551


In [324]:
lr = Ridge()
lr.fit(train_x, train_y)
error(valid_y, lr.predict(valid_x))

0.06188635128608144


##### Сдача первой части соревнования
Загрузите файл `hw_final_closed_data.npy` (ссылка есть на странице с заданием). Если вы используете sklearn-совместимую модель, для генерации посылки вы можете воспользоваться функцией `get_predictions`. В ином случае перепишите функцию для вашей модели и запустите код под следующей ячейкой для генерации посылки.

In [108]:
!wget https://raw.githubusercontent.com/girafe-ai/ml-course/23f_yandex_ml_trainings/homeworks/assignment_final/hw_final_closed_data.npy -O hw_final_closed_data.npy

--2023-11-30 08:32:12--  https://raw.githubusercontent.com/girafe-ai/ml-course/23f_yandex_ml_trainings/homeworks/assignment_final/hw_final_closed_data.npy
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11328 (11K) [application/octet-stream]
Saving to: ‘hw_final_closed_data.npy’

hw_final_closed_dat 100%[===================>]  11.06K  --.-KB/s    in 0s      

2023-11-30 08:32:12 (69.1 MB/s) - ‘hw_final_closed_data.npy’ saved [11328/11328]



In [109]:
assert os.path.exists('hw_final_closed_data.npy'), 'Please, download `hw_final_closed_data.npy` and place it in the working directory'
closed_data = np.load('hw_final_closed_data.npy', allow_pickle=False)

Если необходимо, преобразуйте данные. Преобразованную матрицу объект-признак сохраните в переменную `closed_data`.

In [ ]:
# optional transformations

In [110]:
def get_predictions(model, eval_data, step=10):
    predicted_values = model.predict(eval_data)
    return predicted_values

Обращаем ваше внимание, предсказания округляются до сотых!

In [326]:
predicted_values = np.round(get_predictions(model=lr, eval_data=closed_data), 2)

assert predicted_values.shape == (closed_data.shape[0], ) # predictions should be just one-dimensional array

In [327]:
# do not change the code in the block below
# __________start of block__________
def float_list_to_comma_separated_str(_list):
    _list = list(np.round(np.array(_list), 2))
    return ','.join([str(x) for x in _list])

submission_dict = {
    'predictions': float_list_to_comma_separated_str(predicted_values)
}
with open('submission_dict_final_p01.json', 'w') as iofile:
    json.dump(submission_dict, iofile)

print('File saved to `submission_dict_final_p01.npy`')
# __________end of block__________

File saved to `submission_dict_final_p01.npy`


### Модель №2
Функция `my_transformation` принимает на вход матрицу объект-признак (`numpy.ndarray` типа `np.float`) и преобразует ее в новую матрицу. Данная функция может использовать только numpy-операции, а также арифметические действия.

Для примера доступна функция ниже. Она лишь добавляет новый признак, представляющий собой произведение первого и второго исходных признаков (считая с нуля).

In [414]:
def my_transformation(feature_matrix: np.ndarray):
    new_feature_matrix = np.zeros((feature_matrix.shape[0], feature_matrix.shape[1]+1))
    new_feature_matrix[:, :feature_matrix.shape[1]] = feature_matrix
    new_feature_matrix[:, -1] = np.exp(feature_matrix[:, 0])
    return new_feature_matrix

In [415]:
transformed_train_x = my_transformation(train_x)
transformed_valid_x = my_transformation(valid_x)

In [416]:
transformed_train_x.shape

(640, 8)

In [417]:
lr = Ridge()
lr.fit(transformed_train_x, train_y)

print(
    f'train mse =\t {mean_squared_error(lr.predict(transformed_train_x), train_y):.5f}',
    f'validation mse = {mean_squared_error(lr.predict(transformed_valid_x), valid_y):.5f}',
    sep='\n'
)

train mse =	 0.09763
validation mse = 0.06189


Обращаем ваше внимание, что параметры линейной модели будут округляться до __четырех знаков после запятой__. Это не должно сильно повлиять на качество предсказаний:

In [418]:
original_predictions = lr.predict(transformed_train_x)
rounded_predictions = transformed_train_x.dot(np.round(lr.coef_, 4)) + np.round(lr.intercept_, 4)


assert np.allclose(original_predictions, rounded_predictions, atol=1e-3)

Параметры вашей модели:

In [419]:
w_list = list(np.round(lr.coef_, 4))
print(f'w = {list(np.round(lr.coef_, 4))}\nb = {np.round(lr.intercept_, 4)}')

w = [0.0104, 0.4896, 0.0, 0.5584, 0.0669, 1.4436, 0.0, 0.0059]
b = 1.4227


Напоминаем, ваша модель не должна использовать более 15 параметров (14 весов плюс свободный член).

In [401]:
assert len(w_list) + 1 <= 15

##### Сдача второй части соревнования
Для сдачи вам достаточно отправить функцию `my_transformation` и параметры вашей модели в контест в задачу №2. Пример посылки доступен ниже. Имортирование `numpy` также необходимо.

In [402]:
# __________example_submission_start__________
import numpy as np
def my_transformation(feature_matrix: np.ndarray):
    new_feature_matrix = np.zeros((feature_matrix.shape[0], feature_matrix.shape[1]+1))
    new_feature_matrix[:, :feature_matrix.shape[1]] = feature_matrix
    new_feature_matrix[:, -1] = feature_matrix[:, 0
    ] * feature_matrix[:, 1]
    return new_feature_matrix

w_submission = [-0.0027, -0.2637, 0.0, -0.1134, -0.0165, -0.9329, 0.0, 0.1293]
b_submission = 1.1312
# __________example_submission_end__________

На этом задание завершено. Поздравляем!